In [144]:
#package loading 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
from scipy.stats import percentileofscore
from tqdm import tqdm
import cudf
import numba as nb
import os
import sys
import time
import pickle

In [66]:
from utility import *
from preprocess import * 

In [50]:
import warnings
warnings.filterwarnings("ignore")

# Read datasets

In [4]:
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_database"

In [5]:
# covid database
diagnosis_cov = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_cov = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_cov = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])

/tmp/ipykernel_79664/3208235642.py:2: DtypeWarning: Columns (7,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  diagnosis_cov = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3208235642.py:3: DtypeWarning: Columns (2,3,5,6,9,10,12,13,15,17,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_result_cm_cov = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3208235642.py:4: DtypeWarning: Columns (6,7,9,10,13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  obs_gen_cov = pd.read_csv(data_wd+"/OBS_GEN.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3208235642.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  procedure_cov = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])


In [6]:
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/main_database"

In [7]:
# general database
diagnosis_m = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_m = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_m = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])

/tmp/ipykernel_79664/3211017016.py:2: DtypeWarning: Columns (7,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  diagnosis_m = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3211017016.py:3: DtypeWarning: Columns (9,10,12,13,17,21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_result_cm_m = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3211017016.py:4: DtypeWarning: Columns (6,7,9,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  obs_gen_m = pd.read_csv(data_wd+"/OBS_GEN.csv",header=0,skiprows=[1])
/tmp/ipykernel_79664/3211017016.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  procedure_m = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])


In [8]:
print(diagnosis_m.columns == diagnosis_cov.columns)
print(procedure_m.columns == procedure_cov.columns)
print(demographics_m.columns == demographics_cov.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True]


In [9]:
# union them
diagnosis = pd.concat([diagnosis_m,diagnosis_cov])
print("complete diagnosis")
procedure = pd.concat([procedure_m,procedure_cov])
print("complete procedure")
demographics = pd.concat([demographics_m,demographics_cov])
print("complete demographics")

complete diagnosis
complete lab_result_cm
complete obs_gen
complete procedure
complete demographics


In [10]:
#release memory
del(diagnosis_m)
del(diagnosis_cov)
del(procedure_m)
del(procedure_cov)
del(demographics_m)
del(demographics_cov)

# Extract COVID patients

## import index date and cohort definition

In [11]:
#pasc and covid inex dates
index_date = pd.read_csv('/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_pasc_cohort.csv')

In [12]:
index_date['group'].value_counts()

cont    224041
case      5651
Name: group, dtype: int64

In [13]:
index_date.head()

,ID,index_date,pasc_onset_date,group
0,19Oct2022covidpasc00003601,2022-01-13,NaN,cont
1,19Oct2022covidpasc00003602,2020-09-30,NaN,cont
2,19Oct2022covidpasc00003603,2020-06-15,NaN,cont
3,19Oct2022covidpasc00003604,2020-07-07,NaN,cont
4,19Oct2022covidpasc00003605,2021-01-13,NaN,cont


## choose those with covid19 index date

In [14]:
#choose those with covid19 index date
index_date = index_date.loc[~index_date['index_date'].isna()]

In [15]:
index_date['group'].value_counts()

cont    224041
case      1937
Name: group, dtype: int64

## choose those with pasc date after covid date

In [16]:
index_date['index_date'] = pd.to_datetime(index_date['index_date'])
index_date['pasc_onset_date'] = pd.to_datetime(index_date['pasc_onset_date'])

In [17]:
index_date = index_date.loc[(index_date['index_date']<index_date['pasc_onset_date'])|index_date['pasc_onset_date'].isna()]

In [21]:
index_date['group'].value_counts()

cont    224041
case      1706
Name: group, dtype: int64

# Demographics preprocessing

## drop duplicate

In [19]:
# This is because there are two database to be merged, i.e. covid database and general databse. The medical records are not duplicates abut the patient demographics are.
print(demographics.shape)
demographics = demographics.drop_duplicates(['ID'])
print(demographics.shape)

(455995, 12)
(229689, 12)


## 1st filter by index date cohorts

In [20]:
#pair demographics and index date
print(demographics.shape)
demographics = pd.merge(demographics,index_date,left_on='ID',right_on='ID',how='inner')
demographics = demographics[['ID', 'index_date','pasc_onset_date']]
print(demographics.shape)

(229689, 12)
(225744, 3)


# Extract cancer patient cohort by ICD codes

## All cancer

In [53]:
dx_codes_icd9_all = tuple(['140','141','142','143','144','145','146','147','148','149',
                     '150','151','152','153','154','155','156','157','158','159',
                     '160','161','162','163','164','165','166','167','168','169',
                     '170','171','172','173','174','175','176','177','178','179',
                     '180','181','182','183','184','185','186','187','188','189',
                     '190','191','192','193','194','195','196','197','198','199',
                     '200','201','202','203','204','205','206','207','208','209',
                     '2300','2310','V1001','V1002','V1021','2301', 'V1003','2302', 
                     'V1004','2303','V1005','2304','2305','2306','V1006','2308', 
                     'V1007','2312','V1011','2330','V103','2332','V1042','2331', 
                     '7950','V1041','V1043','2334', 'V1046','V1047', '2337', 'V1051', 
                     'V1052', 'V1085', 'V1086', 'V1087', 'V1072', 'V1071', 'V1079',
                    'V1060', 'V1061', 'V1062', 'V1063', 'V1069','173','196','197','198'])

In [ ]:
dx_codes_icd10_all = tuple(['C'])

In [77]:
#diagnosis
pid_dx_all_diag = extract_cancer_pid(df=diagnosis,\
                                     extract_from_col="DX",\
                                     code_type_col="DX_TYPE",\
                                     codes_icd9=dx_codes_icd9_all,\
                                     codes_icd10=dx_codes_icd10_all,\
                                     subtype="Any")

#procedure
pid_dx_all_procedure = extract_cancer_pid(df=procedure,\
                                          extract_from_col="PX",\
                                          code_type_col="PX_TYPE",\
                                          codes_icd9=dx_codes_icd9_all,\
                                          codes_icd10=dx_codes_icd10_all,\
                                          subtype="Any")

pid_dx_all = pd.concat([pid_dx_all_diag,pid_dx_all_procedure]).\
                drop_duplicates(ignore_index=True)

## Head and neck cancer

In [83]:
dx_codes_icd9_Headandneckcancer = tuple(['1400', '1401', '1403', '1404', '1405', '1406', '1408', '1409', '1410', 
                                         '1411', '1412', '1413', '1414', '1415', '1416', '1418', '1419', '1420', 
                                         '1421', '1422', '1428', '1429', '1430', '1431', '1438', '1439', '1440', 
                                         '1441', '1448', '1449', '1450', '1451', '1452', '1453', '1454', '1455', 
                                         '1456', '1458', '1459', '1460', '1461', '1462', '1463', '1464', '1465', 
                                         '1466', '1467', '1468', '1469', '1470', '1471', '1472', '1473', '1478', 
                                         '1479', '1480', '1481', '1482', '1483', '1488', '1489', '1490', '1491', 
                                         '1498', '1499', '1600', '1601', '1602', '1603', '1604', '1605', '1608', 
                                         '1609', '1610', '1611', '1612', '1613', '1618', '1619', '1950', '2300', 
                                         '2310', 'V1001', 'V1002', 'V1021'])

dx_codes_icd10_Headandneckcancer = tuple(['C760','C00','C01','C02','C03','C04','C05','C06','C07','C08','C09',
                                          'C10','C11','C12','C13','C14','C30','C31','C32'])

## Esophageal cancer

In [85]:
dx_codes_icd9_Esophagealcancer = tuple(['1500', '1501', '1502', '1503', '1504', '1505', '1508', '1509', '2301', 'V1003'])
dx_codes_icd10_Esophagealcancer = tuple(['C15'])

## Stomach cancer

In [86]:
dx_codes_icd9_Stomachcancer = tuple(['1510', '1511', '1512', '1513', '1514', '1515', '1516', '1518', '1519', '2302', 'V1004'])
dx_codes_icd10_Stomachcancer = tuple(['C16'])

## Small intestine

In [87]:
dx_codes_icd9_Smallintestine = tuple(['1520','1521','1522','1523','1528','1529'])
dx_codes_icd10_Smallintestine = tuple(['C17'])

## Colorectal cancer

In [88]:
dx_codes_icd9_Colorectalcancer = tuple(['1530', '1531', '1532', '1533', '1534', '1535', 
                                        '1536', '1537', '1538', '1539', '1590', '2303', 
                                        'V1005', '1540', '1541', '1542', '1543', '1548', 
                                        '2304', '2305', '2306', 'V1006'])
dx_codes_icd10_Colorectalcancer = tuple(['C18','C19','C20','C21'])

## Liver cancer

In [89]:
dx_codes_icd9_Livercancer = tuple(['1550', '1551', '1552', '2308', 'V1007'])
dx_codes_icd10_Livercancer = tuple(['C22'])

## Pancreatic cancer

In [90]:
dx_codes_icd9_Pancreaticcancer = tuple(['1570', '1571', '1572', '1573', '1574', '1578', '1579'])
dx_codes_icd10_Pancreaticcancer = tuple(['C25'])

## Lung caner

In [91]:
dx_codes_icd9_Lungcaner = tuple(['1622', '1623', '1624', '1625', '1628', '1629', '2312', 'V1011'])
dx_codes_icd10_Lungcaner = tuple(['C34'])

## Breast cancer

In [92]:
dx_codes_icd9_Breastcancer = tuple(['1740', '1741', '1742', '1743', '1744', '1745', '1746', '1748', '1749', '1750', '1759', '2330', 'V103'])
dx_codes_icd10_Breastcancer = tuple(['C50'])

## Uteral cancer

In [115]:
dx_codes_icd9_Uteralcancer = tuple(['179', '1820', '1821', '1828', '2332', 'V1042'])
dx_codes_icd10_Uteralcancer = tuple(['C66'])

## Cervical cancer

In [95]:
dx_codes_icd9_Cervicalcancer = tuple(['1800', '1801', '1808', '1809', '2331', '7950', 'V1041'])
dx_codes_icd10_Cervicalcancer = tuple(['C53'])

## Ovarian cancer

In [96]:
dx_codes_icd9_Ovariancancer = tuple(['1830', 'V1043'])
dx_codes_icd10_Ovariancancer = tuple(['C56'])

## Prostate cancer

In [97]:
dx_codes_icd9_Prostatecancer = tuple(['185', '2334', 'V1046'])
dx_codes_icd10_Prostatecancer = tuple(['C61'])

## Testicular cancer

In [98]:
dx_codes_icd9_Testicularcancer = tuple(['1860', '1869', 'V1047'])
dx_codes_icd10_Testicularcancer = tuple(['C62'])

## Bladder cancer

In [99]:
dx_codes_icd9_Bladdercancer = tuple(['1880', '1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '2337', 'V1051'])
dx_codes_icd10_Bladdercancer = tuple(['C67'])

## Kidney cancer

In [100]:
dx_codes_icd9_Kidneycancer = tuple(['1890', '1891', 'V1052'])
dx_codes_icd10_Kidneycancer = tuple(['C64'])

## Brain cancer

In [101]:
dx_codes_icd9_Braincancer = tuple(['1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918',
                                   '1919', '1920', '1921', '1922', '1923', '1928', '1929', 'V1085', 'V1086'])
dx_codes_icd10_Braincancer = tuple(['C71'])

## Thyroid cancer

In [102]:
dx_codes_icd9_Thyroidcancer = tuple(['193', 'V1087'])
dx_codes_icd10_Thyroidcancer = tuple(['C73'])

## Hodgkin's lymphoma

In [103]:
dx_codes_icd9_Hodgkinslymphoma = tuple(['20100', '20101', '20102', '20103', '20104', '20105', '20106', '20107',
                                        '20108', '20110', '20111', '20112', '20113', '20114', '20115', '20116',
                                        '20117', '20118', '20120', '20121', '20122', '20123', '20124', '20125', 
                                        '20126', '20127', '20128', '20140', '20141', '20142', '20143', '20144',
                                        '20145', '20146', '20147', '20148', '20150', '20151', '20152', '20153', 
                                        '20154', '20155', '20156', '20157', '20158', '20160', '20161', '20162', 
                                        '20163', '20164', '20165', '20166', '20167', '20168', '20170', '20171',
                                        '20172', '20173', '20174', '20175', '20176', '20177', '20178', '20190',
                                        '20191', '20192', '20193', '20194', '20195', '20196', '20197', '20198', 'V1072'])
dx_codes_icd10_Hodgkinslymphoma = tuple(['C81'])

## Non-Hodgkin`s lymphoma

In [104]:
dx_codes_icd9_NonHodgkinslymphoma = tuple(['20000', '20001', '20002', '20003', '20004', '20005', '20006', '20007', 
                                           '20008', '20010', '20011', '20012', '20013', '20014', '20015', '20016', 
                                           '20017', '20018', '20020', '20021', '20022', '20023', '20024', '20025', 
                                           '20026', '20027', '20028', '20080', '20081', '20082', '20083', '20084', 
                                           '20085', '20086', '20087', '20088', '20200', '20201', '20202', '20203', 
                                           '20204', '20205', '20206', '20207', '20208', '20210', '20211', '20212', 
                                           '20213', '20214', '20215', '20216', '20217', '20218', '20220', '20221', 
                                           '20222', '20223', '20224', '20225', '20226', '20227', '20228', '20280', 
                                           '20281', '20282', '20283', '20284', '20285', '20286', '20287', '20288', 
                                           '20290', '20291', '20292', '20293', '20294', '20295', '20296', '20297', 
                                           '20298', 'V1071', 'V1079'])
dx_codes_icd10_NonHodgkinslymphoma = tuple(['C82','C83','C84','C85'])

## Leukemias

In [106]:
dx_codes_icd9_Leukemias = tuple(['20240', '20241', '20242', '20243', '20244', '20245', '20246', '20247',
                                 '20248', '2031', '20310', '20311', '2040', '20400', '20401', '2041', '20410', '20411', '2042', 
                                 '20420', '20421', '2048', '20480', '20481', '2049', '20490', '20491', '2050', '20500', '20501', 
                                 '2051', '20510', '20511', '2052', '20520', '20521', '2053', '20530', '20531', '2058', '20580', 
                                 '20581', '2059', '20590', '20591', '2060', '20600', '20601', '2061', '20610', '20611', '2062', 
                                 '20620', '20621', '2068', '20680', '20681', '2069', '20690', '20691', '2070', '20700', '20701', 
                                 '2071', '20710', '20711', '2072', '20720', '20721', '2078', '20780', '20781', '2080', '20800', 
                                 '20801', '2081', '20810', '20811', '2082', '20820', '20821', '2088', '20880', '20881', '2089', 
                                 '20890', '20891', 'V1060', 'V1061', 'V1062', 'V1063', 'V1069'])
dx_codes_icd10_Leukemias = tuple(['C91','C92','C93','C94','C95'])

## Multiple myeloma

In [107]:
dx_codes_icd9_Multiplemyeloma = tuple(['2030', '20300', '20301', '2038', '20380', '20381'])
dx_codes_icd10_Multiplemyeloma = tuple(['C90'])

## Skin cancer

In [108]:
dx_codes_icd9_Skincancer = tuple(['172','173'])
dx_codes_icd10_Skincancer = tuple(['C43','C44','C4A'])

## Secondary cancer

In [109]:
dx_codes_icd9_Secondarycancer = tuple(['196', '197', '198'])
dx_codes_icd10_Secondarycancer = tuple(['C77', 'C78', 'C79', 'C7B'])

## Assemble them

In [117]:
icd9_list = [dx_codes_icd9_Headandneckcancer,
             dx_codes_icd9_Esophagealcancer,
             dx_codes_icd9_Stomachcancer,
             dx_codes_icd9_Smallintestine,
             dx_codes_icd9_Colorectalcancer,
             dx_codes_icd9_Livercancer,
             dx_codes_icd9_Pancreaticcancer,
             dx_codes_icd9_Lungcaner,
             dx_codes_icd9_Breastcancer,
             dx_codes_icd9_Uteralcancer,
             dx_codes_icd9_Cervicalcancer,
             dx_codes_icd9_Ovariancancer,
             dx_codes_icd9_Prostatecancer,
             dx_codes_icd9_Testicularcancer,
             dx_codes_icd9_Bladdercancer,
             dx_codes_icd9_Kidneycancer,
             dx_codes_icd9_Braincancer,
             dx_codes_icd9_Thyroidcancer,
             dx_codes_icd9_Hodgkinslymphoma,
             dx_codes_icd9_NonHodgkinslymphoma,
             dx_codes_icd9_Leukemias,
             dx_codes_icd9_Multiplemyeloma,
             dx_codes_icd9_Skincancer,
             dx_codes_icd9_Secondarycancer
            ]

icd10_list = [dx_codes_icd10_Headandneckcancer,
             dx_codes_icd10_Esophagealcancer,
             dx_codes_icd10_Stomachcancer,
             dx_codes_icd10_Smallintestine,
             dx_codes_icd10_Colorectalcancer,
             dx_codes_icd10_Livercancer,
             dx_codes_icd10_Pancreaticcancer,
             dx_codes_icd10_Lungcaner,
             dx_codes_icd10_Breastcancer,
             dx_codes_icd10_Uteralcancer,
             dx_codes_icd10_Cervicalcancer,
             dx_codes_icd10_Ovariancancer,
             dx_codes_icd10_Prostatecancer,
             dx_codes_icd10_Testicularcancer,
             dx_codes_icd10_Bladdercancer,
             dx_codes_icd10_Kidneycancer,
             dx_codes_icd10_Braincancer,
             dx_codes_icd10_Thyroidcancer,
             dx_codes_icd10_Hodgkinslymphoma,
             dx_codes_icd10_NonHodgkinslymphoma,
             dx_codes_icd10_Leukemias,
             dx_codes_icd10_Multiplemyeloma,
             dx_codes_icd10_Skincancer,
             dx_codes_icd10_Secondarycancer
            ]

subtype_list = ['Head and neck',
                'Esophageal',
                'Stomach',
                'Small intestine',
                'Colorectal',
                'Liver',
                'Pancretic',
                'Lung',
                'Breast',
                'Uteral',
                'Cervical',
                'Ovarian',
                'Prostate',
                'Testicular',
                'Bladder',
                'Kidney',
                'Brain',
                'Thyroid',
                'Hodgkin\'s lymphoma',
                'Non-Hodgkin\'s lymphoma',
                'Leukemias',
                'Multiple myeloma',
                'Skin',
                'Secondar cancer']

In [135]:
pid_dx_subtypes = pid_dx_all.loc[pid_dx_all['ID']==""].reset_index(drop=True)

print("-----------------tagging cancer subtypes, please wait:")
start_time = time.time()
for indx, sub in enumerate(subtype_list):
    dx_codes_icd9_tmp = icd9_list[indx]
    dx_codes_icd10_tmp = icd10_list[indx]
    
    #diagnosis
    pid_dx_subtypes_tmp = extract_cancer_pid(df=pid_dx_all,\
                                         extract_from_col="DX",\
                                         code_type_col="DX_TYPE",\
                                         codes_icd9=dx_codes_icd9_tmp,\
                                         codes_icd10=dx_codes_icd10_tmp,\
                                         subtype=sub)

    pid_dx_subtypes = pd.concat([pid_dx_subtypes,pid_dx_subtypes_tmp]).\
                                drop_duplicates(ignore_index=True)
    
print("Done in --- %s seconds ---" % (time.time() - start_time))

-----------------tagging cancer subtypes, please wait:
Done in --- 15.189109325408936 seconds ---


## Other

In [138]:
pid_dx_subtypes_other = pid_dx_all.loc[~is_in_set_pnb(pid_dx_all['ID'],pid_dx_subtypes['ID'])]
pid_dx_subtypes_other['subtype']='Other'

## Finalizing

In [139]:
pid_dx_subtypes = pd.concat([pid_dx_subtypes,pid_dx_subtypes_other])

In [142]:
#checksum ID
if len(set(pid_dx_subtypes['ID'])) == len(set(pid_dx_all['ID'])):
    print("Pass")
else:
    print("Fail")

Pass


# write

In [146]:
pickle.dump(pid_dx_subtypes,open('/data/datasets/changyuyin/2_COVIDwithCancer/pid_dx_cancer_subtypes.pkl','wb'))